**извлечение признаков из текста на естественном языке**

Евгений Борисов borisov.e@solarl.ru

In [14]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

import sys
import re
import gzip
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [10]:
pd.options.display.max_colwidth = 200  

In [11]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
1637,"09:42 - 13.12.2016\n\nУченые прогнозируют мощное извержение вулкана Безымянный на Камчатке, которое произойдет в ближайшее время.\n\nВулкан Безымянный может представлять опасность для местных и ме...",science
1553,"Спецслужбы предотвратили 42 преступления террористической направленности в этом году, в том числе в Москве, Санкт-Петербурге, Екатеринбурге и Нижнем Новгороде, сообщил глава ФСБ РФ Александр Бортн...",politics
1536,"13 декабря 2016, 10:07\n\nМВД выявило хищение 44 млн рублей, выделенных на реконструкцию центра им. Сербского в Москве. Об этом «Росбалту» сообщила официальный представитель МВД России Ирина Волк....",incident
2972,"Дата: 11.12.2016 16:45:13\n\nАвторынок в ноябре «пошел на взлет» после 2 лет депрессии\n\nКомментируя данные по продажам автомобилей в России в ноябре этого года, председатель Комитета автопроизво...",auto


In [12]:
data = df['text'].tolist()

In [29]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

print('количество стоп-слов:',len(stopwords))

количество стоп-слов: 151


In [31]:
# stopwords

In [32]:
# очистка текста 
def preprocessor(tt):
    tt = [ t.lower() for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'\W', ' ', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ re.sub( r'_', ' ', t)  for t in tt ] 
    tt = [ re.sub( r'\b\d+\b', ' ', t ) for t in tt ] # замена цифр
    tt = [ re.sub( r'\b.*\d+.*\b', ' ', t ) for t in tt ]  # замена буквенно-цифровых кодов
    tt = [ re.sub( r'[a-z]+', ' ', t ) for t in tt ]  # удаление слов из символов латиницы
    return [ t.strip() for t in tt if len(t.strip())>2 ] # удаление коротких слов

In [33]:
%%time 

df['text_clean'] = df['text']

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() ] )

# очистка от лишних символов
df['text_clean'] = df['text_clean'].apply(lambda t: preprocessor(t) )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 3.37 s, sys: 42.3 ms, total: 3.41 s
Wall time: 3.41 s


In [38]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t).strip())

In [34]:
%xdel stopwords

In [39]:
df.sample(10)

,text,tag,text_clean
1735,"Во вторник утром, 13 декабря, мировые цены на нефть пошли вниз после достижения днем ранее максимальных отметок с июля 2015 года.\n\nТак, к 08:27 мск стоимость февральских фьючерсов на «черное зол...",economics,вторник утром декабря мировые цены нефть пошли вниз достижения днем ранее максимальных отметок июля года мск стоимость февральских фьючерсов черное золото марки снизилась доллара бочку цена фьючер...
849,"Неофициальное продолжение фильма ""1+1"" в Минске выходит на месяц раньше, чем в России\n\n8 декабря 2016 в 23:06\n\nАртем Бордовский, AFISHA.TUT.BY\n\nФранцузскую трагикомедию «2+1», в которой снял...",culture,неофициальное продолжение фильма минске выходит месяц раньше россии декабря артем бордовский французскую трагикомедию которой снялся омар увидеть кинотеатрах минска декабря фильм появился столично...
888,"""Гамлета"" с Бенедиктом Камбербетчем в последний раз покажут в Минске\n\n28 ноября 2016 в 22:00\n\nAFISHA.TUT.BY\n\nВ преддверии новогодних праздников с 6 декабря в столичных кинотеатрах стартуют п...",culture,гамлета бенедиктом камбербетчем последний покажут минске ноября преддверии новогодних праздников декабря столичных кинотеатрах стартуют показы проекта программе постановки широкой аудитории сказоч...
1114,"Киевское ""Динамо"" Корзуна забило 6 голов в матче Лиги чемпионов, Калачев сыграет в Лиге Европы\n\n7 декабря 2016 в 2:45\n\nSPORT.TUT.BY\n\nВо вторник состоялись последние матчи группового этапа Ли...",sport,киевское динамо корзуна забило голов матче лиги чемпионов калачев сыграет лиге европы декабря вторник состоялись последние матчи группового этапа лиги чемпионов группах фото ростов тимофея калачев...
375,"Идентифицирован белок, уничтожающий бациллу сибирской язвы Немецким\nученым из Института инфекционной биологии им. Макса Планка удалось\nидентифицировать белок, играющий ключевую роль в подавлении...",health,идентифицирован белок уничтожающий бациллу сибирской язвы немецким ученым института инфекционной биологии макса планка удалось идентифицировать белок играющий ключевую роль подавлении спор сибирск...
2703,"Разработчик ""Т-Платформы"" утверждает, что ноутбук сохранит работоспособность после сильных ударов, воздействия воды и критических для обычной электроники температур: от -50 до +50 градусов.\n\nВ к...",tech,разработчик платформы утверждает ноутбук сохранит работоспособность сильных ударов воздействия воды критических обычной электроники температур градусов компании намерены создать сразу варианта ноу...
2479,Новый проект Бориса Гребенщикова с симфоническим оркестром услышат в Томске. Концерт состоится 13 декабря и его с нетерпением ожидают поклонники творчества группы «Аквариум» и ее солиста- легендар...,culture,новый проект бориса гребенщикова симфоническим оркестром услышат томске концерт состоится декабря нетерпением ожидают поклонники творчества группы аквариум солиста легендарного бориса гребенщикова...
2003,"Промоутер Александра Поветкина Андрей Рябинский подчеркнул, что все допинг-пробы российского боксера, взятые перед боем за звание ""временного"" чемпиона мира по версии WBC с Бермейном Стиверном, да...",sport,промоутер александра поветкина андрей рябинский подчеркнул допинг пробы российского боксера взятые боем звание временного чемпиона мира версии бермейном стиверном дали отрицательный результат поед...
1061,"""У людей нет денег"". В райцентрах одежду на рынках покупают под расписку\n\n29 ноября 2016 в 13:40\n\nNaviny.by\n\nНа фоне существенного падения доходов жители небольших белорусских городов все ча...",social,людей денег райцентрах одежду рынках покупают расписку ноября фоне существенного падения доходов жители небольших белорусских городов чаще приобретают одежду рынках рассрочку расписку снимок носит...
1084,"Грабители обворовали дом футболиста ""Реала"", пока он играл в матче Лиги чемпионов\n\n9 декабря 2016 в 13:13\n\nТАСС\n\nРафаэль Варан. Фото: Дарья Бурякина, TUT.BY\n\nГруппа грабителей проникла в д...",sport,грабители обворовали дом футб

---

In [3]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [40]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features,
                                 stop_words=stop_words_russian )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [41]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)


LDA:

Тема 0: народов коренных севера версия малочисленных установлена внимание
Тема 1: году года рублей млрд фильм долларов тысяч
Тема 2: года компании также декабря компания году рублей
Тема 3: ученые руб также дом сети специалисты дома
Тема 4: декабря дтп области человек района результате водитель
Тема 5: россии также декабря года заявил ранее глава
Тема 6: это беларуси которые время лет очень могут
Тема 7: сша трамп президент трампа президента дональд декабря
Тема 8: это очень который просто человека человек которые
Тема 9: динамо мяч мира лучший декабря стал чемпионата


---

In [42]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features, 
                                   stop_words=stop_words_russian )

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [43]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )


NMF(Frobenius norm):

Тема 0: это которые ученые лет очень время декабря
Тема 1: трамп сша трампа дональд президент избранный президента
Тема 2: дтп водитель результате области мвд декабря происшествия
Тема 3: савченко украины партии надежда заявила лидер действия
Тема 4: народов севера коренных малочисленных края фестиваль июля
Тема 5: динамо чемпионата матче мира очков матча лиги
Тема 6: рублей года компания млн компании году млрд
Тема 7: версия браузер проигрывателя старая отключен установлена поддерживает
Тема 8: россии путин президент президента заявил глава министр
Тема 9: алеппо города боевиков сирии жителей восточной сутки


---

In [44]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )


NMF(generalized Kullback-Leibler divergence):

Тема 0: это фото декабря время которые также года
Тема 1: сша ранее президент трамп президента трампа также
Тема 2: декабря результате области сообщили сообщает человек пресс
Тема 3: это сегодня который украины декабря своей день
Тема 4: также сообщил рамках области россии рублей числе
Тема 5: декабря стал мира место чемпионата который года
Тема 6: также года компания компании россии году ранее
Тема 7: установлена версия поддерживает внимание отключен видео старая
Тема 8: россии декабря заявил президент страны фото президента
Тема 9: ученые сообщает также новости ранее специалисты риа
